In [1]:
import os
import json
from pyvi import ViTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn import metrics 
from sklearn.naive_bayes import GaussianNB
import numpy as np


In [ ]:
def preprocess(data_path, stopword_path, dims):
    data = []
    target = []
    target_names = []

    # Duyệt qua từng file trong thư mục
    for file_name in os.listdir(data_path):
        if file_name.endswith(".json"):  # Chỉ xử lý file .json
            # Đường dẫn file
            file_path = os.path.join(data_path, file_name)
            # Lấy nhãn từ tên file (bỏ phần .json)
            category = os.path.splitext(file_name)[0]

            # Thêm category vào danh sách nhãn nếu chưa có
            if category not in target_names:
                target_names.append(category)

            # Đọc file JSON
            with open(file_path, "r", encoding="utf-8") as file:
                json_data = json.load(file)
                for item in json_data:
                    content = item.get("content", "").strip()
                    if content.lower() == "none":  # Loại bỏ content = "None"
                        continue
                    # Ghép nội dung từ title và content
                    text = (item.get("title", "") + " " + item.get("content", "")).strip()
                    data.append(text)
                    # Thêm chỉ số của category vào target
                    target.append(target_names.index(category))
                    
    # Đọc danh sách từ dừng
    with open(stopword_path, encoding="utf-8") as f:
         stopwords = f.readlines()
    stopwords = [x.strip().replace(" ", "_") for x in stopwords]

    # Tách từ tiếng Việt
    data = [ViTokenizer.tokenize(sample) for sample in data]

    # Chuyển văn bản thành biểu diễn TF-IDF vector
    tfidf_vectorizer = TfidfVectorizer(stop_words=stopwords)
    data_preprocessed = tfidf_vectorizer.fit_transform(data)
    
    # Chuyển dữ liệu TF-IDF thành ma trận
    tfidf_mat = data_preprocessed.toarray()

    # Áp dụng PCA để giảm chiều dữ liệu
    pca = PCA(n_components=dims)
    x_reduced = pca.fit_transform(tfidf_mat)

    return x_reduced, np.array(target), target_names, tfidf_vectorizer





In [ ]:
# Sử dụng hàm
data_path = "json"
stopword_path = "vietnamese-stopwords.txt"
dims = 256

In [ ]:
# Tiền xử lý dữ liệu và trả về vectorizer
X_data, y_data, target_names, tfidf_vectorizer = preprocess(data_path, stopword_path, dims)
print(f"Dữ liệu đã tiền xử lý: {X_data.shape}")
print(f"Số nhãn: {len(target_names)}")
print("Danh sách nhãn và ID tương ứng:")
for idx, name in enumerate(target_names):
    print(f"ID: {idx}, Nhãn: {name}")


In [ ]:

def train_model(classifier, X_data, y_data, target_names):
    X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=42, stratify=y_data)
    classifier.fit(X_train, y_train)
    
    # Dự đoán trên tập train và test
    train_predictions = classifier.predict(X_train)
    test_predictions = classifier.predict(X_test)

    # Đánh giá độ chính xác
    train_accuracy = metrics.accuracy_score(y_train, train_predictions)
    test_accuracy = metrics.accuracy_score(y_test, test_predictions)

    print(f"Train accuracy: {train_accuracy:.2f}")
    print(f"Test accuracy: {test_accuracy:.2f}")

    print("\nBáo cáo phân loại trên tập test:")
    print(metrics.classification_report(y_test, test_predictions, target_names=target_names))

In [ ]:
def predict_text(classifier, tfidf_vectorizer, text, stopwords):
    # Tách từ
    tokenized_text = ViTokenizer.tokenize(text)
    print("Câu sau khi tách từ:", tokenized_text)

    # Chuyển văn bản thành vector TF-IDF
    tfidf_vector = tfidf_vectorizer.transform([tokenized_text])
    tfidf_mat = tfidf_vector.toarray()

    # Dự đoán nhãn
    prediction = classifier.predict(tfidf_mat)
    return prediction

In [ ]:
from sklearn.naive_bayes import GaussianNB

# Tạo và huấn luyện mô hình Naive Bayes
naive_bayes_classifier = GaussianNB()
train_model(naive_bayes_classifier, X_data, y_data, target_names)

# Văn bản cần dự đoán
input_text = "Ca sĩ Hiếu Thứ Hai ra MV mới"

# Dự đoán nhãn
predicted_label = predict_text(naive_bayes_classifier, tfidf_vectorizer, input_text, stopword_path)

# Hiển thị nhãn tương ứng
print(f"Dự đoán nhãn: {target_names[predicted_label[0]]}")


In [2]:
def preprocess(data_path, stopword_path, dims):
    data = []
    target = []
    target_names = []

    # Duyệt qua từng file trong thư mục
    for file_name in os.listdir(data_path):
        if file_name.endswith(".json"):  # Chỉ xử lý file .json
            # Đường dẫn file
            file_path = os.path.join(data_path, file_name)
            # Lấy nhãn từ tên file (bỏ phần .json)
            category = os.path.splitext(file_name)[0]

            # Thêm category vào danh sách nhãn nếu chưa có
            if category not in target_names:
                target_names.append(category)

            # Đọc file JSON
            with open(file_path, "r", encoding="utf-8") as file:
                json_data = json.load(file)
                for item in json_data:
                    content = item.get("content", "").strip()
                    if content.lower() == "none":  # Loại bỏ content = "None"
                        continue
                    # Ghép nội dung từ title và content
                    text = (item.get("title", "") + " " + item.get("content", "")).strip()
                    data.append(text)
                    # Thêm chỉ số của category vào target
                    target.append(target_names.index(category))
                    
    # Đọc danh sách từ dừng
    with open(stopword_path, encoding="utf-8") as f:
         stopwords = f.readlines()
    stopwords = [x.strip().replace(" ", "_") for x in stopwords]

    # Tách từ tiếng Việt
    data = [ViTokenizer.tokenize(sample) for sample in data]

    # Chuyển văn bản thành biểu diễn TF-IDF vector
    tfidf_vectorizer = TfidfVectorizer(stop_words=stopwords)
    data_preprocessed = tfidf_vectorizer.fit_transform(data)
    
    # Chuyển dữ liệu TF-IDF thành ma trận
    tfidf_mat = data_preprocessed.toarray()

    # Áp dụng PCA để giảm chiều dữ liệu
    pca = PCA(n_components=dims)
    x_reduced = pca.fit_transform(tfidf_mat)

    return x_reduced, np.array(target), target_names, tfidf_vectorizer





In [4]:
# Sử dụng hàm
data_path = "json"
stopword_path = "vietnamese-stopwords.txt"
dims = 256

In [ ]:
# Tiền xử lý dữ liệu và trả về vectorizer
X_data, y_data, target_names, tfidf_vectorizer = preprocess(data_path, stopword_path, dims)
print(f"Dữ liệu đã tiền xử lý: {X_data.shape}")
print(f"Số nhãn: {len(target_names)}")
print("Danh sách nhãn và ID tương ứng:")
for idx, name in enumerate(target_names):
    print(f"ID: {idx}, Nhãn: {name}")


In [7]:

def train_model(classifier, X_data, y_data, target_names):
    X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=42, stratify=y_data)
    classifier.fit(X_train, y_train)
    
    # Dự đoán trên tập train và test
    train_predictions = classifier.predict(X_train)
    test_predictions = classifier.predict(X_test)

    # Đánh giá độ chính xác
    train_accuracy = metrics.accuracy_score(y_train, train_predictions)
    test_accuracy = metrics.accuracy_score(y_test, test_predictions)

    print(f"Train accuracy: {train_accuracy:.2f}")
    print(f"Test accuracy: {test_accuracy:.2f}")

    print("\nBáo cáo phân loại trên tập test:")
    print(metrics.classification_report(y_test, test_predictions, target_names=target_names))

In [9]:
def predict_text(classifier, tfidf_vectorizer, text, stopwords):
    # Tách từ
    tokenized_text = ViTokenizer.tokenize(text)
    print("Câu sau khi tách từ:", tokenized_text)

    # Chuyển văn bản thành vector TF-IDF
    tfidf_vector = tfidf_vectorizer.transform([tokenized_text])
    tfidf_mat = tfidf_vector.toarray()

    # Dự đoán nhãn
    prediction = classifier.predict(tfidf_mat)
    return prediction

In [ ]:
from sklearn.naive_bayes import GaussianNB

# Tạo và huấn luyện mô hình Naive Bayes
naive_bayes_classifier = GaussianNB()
train_model(naive_bayes_classifier, X_data, y_data, target_names)

# Văn bản cần dự đoán
input_text = "Ca sĩ Hiếu Thứ Hai ra MV mới"

# Dự đoán nhãn
predicted_label = predict_text(naive_bayes_classifier, tfidf_vectorizer, input_text, stopword_path)

# Hiển thị nhãn tương ứng
print(f"Dự đoán nhãn: {target_names[predicted_label[0]]}")


In [12]:
def preprocess(data_path, stopword_path, dims):
    data = []
    target = []
    target_names = []

    # Duyệt qua từng file trong thư mục
    for file_name in os.listdir(data_path):
        if file_name.endswith(".json"):  # Chỉ xử lý file .json
            # Đường dẫn file
            file_path = os.path.join(data_path, file_name)
            # Lấy nhãn từ tên file (bỏ phần .json)
            category = os.path.splitext(file_name)[0]

            # Thêm category vào danh sách nhãn nếu chưa có
            if category not in target_names:
                target_names.append(category)

            # Đọc file JSON
            with open(file_path, "r", encoding="utf-8") as file:
                json_data = json.load(file)
                for item in json_data:
                    content = item.get("content", "").strip()
                    if content.lower() == "none":  # Loại bỏ content = "None"
                        continue
                    # Ghép nội dung từ title và content
                    text = (item.get("title", "") + " " + item.get("content", "")).strip()
                    data.append(text)
                    # Thêm chỉ số của category vào target
                    target.append(target_names.index(category))

    # Đọc danh sách từ dừng
    with open(stopword_path, encoding="utf-8") as f:
        stopwords = f.readlines()
    stopwords = [x.strip().replace(" ", "_") for x in stopwords]

    # Tách từ tiếng Việt
    data = [ViTokenizer.tokenize(sample) for sample in data]

    # Chuyển văn bản thành biểu diễn TF-IDF vector
    tfidf_vectorizer = TfidfVectorizer(stop_words=stopwords)
    data_preprocessed = tfidf_vectorizer.fit_transform(data)

    # Chuyển dữ liệu TF-IDF thành ma trận dày
    tfidf_mat = data_preprocessed.toarray()

    # Áp dụng PCA để giảm chiều dữ liệu
    pca = PCA(n_components=dims)
    x_reduced = pca.fit_transform(tfidf_mat)

    return x_reduced, np.array(target), target_names, tfidf_vectorizer, pca


def predict_text(classifier, tfidf_vectorizer, pca, text):
    # Tách từ
    tokenized_text = ViTokenizer.tokenize(text)
    print("Câu sau khi tách từ:", tokenized_text)

    # Chuyển văn bản thành vector TF-IDF
    tfidf_vector = tfidf_vectorizer.transform([tokenized_text])
    tfidf_mat = tfidf_vector.toarray()

    # Giảm chiều dữ liệu bằng PCA
    reduced_features = pca.transform(tfidf_mat)

    # Dự đoán nhãn
    prediction = classifier.predict(reduced_features)
    return prediction


# Đường dẫn dữ liệu
data_path = "json"
stopword_path = "vietnamese-stopwords.txt"
dims = 256

# Tiền xử lý dữ liệu và trả về vectorizer cùng PCA
X_data, y_data, target_names, tfidf_vectorizer, pca = preprocess(data_path, stopword_path, dims)
print(f"Dữ liệu đã tiền xử lý: {X_data.shape}")
print(f"Số nhãn: {len(target_names)}")
print("Danh sách nhãn và ID tương ứng:")
for idx, name in enumerate(target_names):
    print(f"ID: {idx}, Nhãn: {name}")

# Tạo và huấn luyện mô hình Naive Bayes
naive_bayes_classifier = GaussianNB()
train_model(naive_bayes_classifier, X_data, y_data, target_names)

# Văn bản cần dự đoán
input_text = "Ca sĩ Hiếu Thứ Hai ra MV mới"

# Dự đoán nhãn
predicted_label = predict_text(naive_bayes_classifier, tfidf_vectorizer, pca, input_text)

# Hiển thị nhãn tương ứng
print(f"Dự đoán nhãn: {target_names[predicted_label[0]]}")


Dữ liệu đã tiền xử lý: (7099, 256)
Số nhãn: 12
Danh sách nhãn và ID tương ứng:
ID: 0, Nhãn: suc_khoe
ID: 1, Nhãn: am_thuc
ID: 2, Nhãn: giai_tri
ID: 3, Nhãn: giao_duc
ID: 4, Nhãn: chinh_tri
ID: 5, Nhãn: phap_luat
ID: 6, Nhãn: du_lich
ID: 7, Nhãn: khoa_hoc
ID: 8, Nhãn: doi_song
ID: 9, Nhãn: cong_nghe
ID: 10, Nhãn: the_thao
ID: 11, Nhãn: giao_thong
Train accuracy: 0.86
Test accuracy: 0.84

Báo cáo phân loại trên tập test:
              precision    recall  f1-score   support

    suc_khoe       0.91      0.82      0.86       120
     am_thuc       0.89      0.93      0.91       109
    giai_tri       0.96      0.92      0.94       120
    giao_duc       0.87      0.90      0.88       119
   chinh_tri       0.81      0.82      0.81       117
   phap_luat       0.62      0.85      0.72       120
     du_lich       0.79      0.82      0.81       119
    khoa_hoc       0.87      0.69      0.77       120
    doi_song       0.67      0.68      0.68       117
   cong_nghe       0.89      0.69   